In [1]:
import numpy as np
import pandas as pd
import operator
import pickle

In [2]:
## Reading MIMIC Data

data_dir = "../Data/"
admissions = pd.read_csv(data_dir+"ADMISSIONS.csv.gz", compression="gzip")
callout = pd.read_csv(data_dir+"CALLOUT.csv.gz", compression="gzip")
care_givers = pd.read_csv(data_dir+"CAREGIVERS.csv.gz", compression="gzip")
#chart_events = pd.read_csv(data_dir+"CHARTEVENTS.csv.gz", compression="gzip")
cpt_events = pd.read_csv(data_dir+"CPTEVENTS.csv.gz", compression="gzip")

/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,5,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
date_time_events = pd.read_csv(data_dir+"DATETIMEEVENTS.csv.gz", compression="gzip")
d_cpt = pd.read_csv(data_dir+"D_CPT.csv.gz", compression="gzip")
diagnoses_icd = pd.read_csv(data_dir+"DIAGNOSES_ICD.csv.gz", compression="gzip")
d_icd_diagnoses = pd.read_csv(data_dir+"D_ICD_DIAGNOSES.csv.gz", compression="gzip")
d_icd_procedures = pd.read_csv(data_dir+"D_ICD_PROCEDURES.csv.gz", compression="gzip")
d_items = pd.read_csv(data_dir+"D_ITEMS.csv.gz", compression="gzip")
d_lab_items = pd.read_csv(data_dir+"D_LABITEMS.csv.gz", compression="gzip")

/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
drug_codes = pd.read_csv(data_dir+"DRGCODES.csv.gz", compression="gzip")
icu_stays = pd.read_csv(data_dir+"ICUSTAYS.csv.gz", compression="gzip")
#input_events_cv = pd.read_csv(data_dir+"INPUTEVENTS_CV.csv.gz", compression="gzip")
#input_events_mv = pd.read_csv(data_dir+"INPUTEVENTS_MV.csv.gz", compression="gzip")
#lab_events = pd.read_csv(data_dir+"LABEVENTS.csv.gz", compression="gzip")
microbiology_events = pd.read_csv(data_dir+"MICROBIOLOGYEVENTS.csv.gz", compression="gzip")
#note_events = pd.read_csv(data_dir+"NOTEEVENTS.csv.gz", compression="gzip")

In [4]:
#output_events = pd.read_csv(data_dir+"OUTPUTEVENTS.csv.gz", compression="gzip")
patients = pd.read_csv(data_dir+"PATIENTS.csv.gz", compression="gzip")
#prescriptions = pd.read_csv(data_dir+"PRESCRIPTIONS.csv.gz", compression="gzip")
procedure_events_mv = pd.read_csv(data_dir+"PROCEDUREEVENTS_MV.csv.gz", compression="gzip")
procedures_icd = pd.read_csv(data_dir+"PROCEDURES_ICD.csv.gz", compression="gzip")
services = pd.read_csv(data_dir+"SERVICES.csv.gz", compression="gzip")
transfers = pd.read_csv(data_dir+"TRANSFERS.csv.gz", compression="gzip")

In [ ]:
prescriptions = pd.read_csv(data_dir+"PRESCRIPTIONS.csv.gz", compression="gzip")
prescriptions.head(10)
#prescri

In [7]:
drugs = prescriptions["DRUG"]
drugs_set = set (drugs)
print (len(drugs_set))
drugs.head(5)

4525


0        Tacrolimus
1          Warfarin
2    Heparin Sodium
3               D5W
4        Furosemide
Name: DRUG, dtype: object

In [8]:
drug_type = prescriptions["DRUG_TYPE"]
drug_type = set (drug_type)
drug_type

{'ADDITIVE', 'BASE', 'MAIN'}

In [9]:
drug_main = prescriptions[prescriptions["DRUG_TYPE"] == "MAIN"]["DRUG"]
drug_main = set (drug_main)
len(drug_main)

drug_base = prescriptions[prescriptions["DRUG_TYPE"] == "BASE"]["DRUG"]
drug_base = set (drug_base)
len(drug_base)

drug_add = prescriptions[prescriptions["DRUG_TYPE"] == "ADDITIVE"]["DRUG"]
drug_add = set (drug_add)
len(drug_add)

68

In [10]:
drugs.head()

0        Tacrolimus
1          Warfarin
2    Heparin Sodium
3               D5W
4        Furosemide
Name: DRUG, dtype: object

In [12]:
print(len(patients))
patients.head()

46520


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [25]:
subjects_alive = patients[patients["EXPIRE_FLAG"] == 0]["SUBJECT_ID"]
#len(patients_alive)
#len(set(patients_alive["SUBJECT_ID"]))
subjects_expired = patients[patients["EXPIRE_FLAG"] == 1]["SUBJECT_ID"]
print ("Alive: %i; Expired: %i" %(len(set(subjects_alive)), len(set(subjects_expired))))

Alive: 30761; Expired: 15759


In [26]:
max_prescribed_drug = prescriptions["DRUG"].value_counts().idxmax()
max_prescribed_drug_count = prescriptions["DRUG"].value_counts().max()
print(max_prescribed_drug, max_prescribed_drug_count)

Potassium Chloride 192993


In [27]:
subjects_with_max_pres_drug = prescriptions[prescriptions["DRUG"] == max_prescribed_drug]["SUBJECT_ID"]
subjects_max_pres_drug_set = set(subjects_with_max_pres_drug)
len(subjects_max_pres_drug_set)

29711

In [29]:
subjects_expired_with_max_pres_drug = set(subjects_expired).intersection(set(subjects_max_pres_drug_set))
subjects_alive_with_max_pres_drug = set(subjects_alive).intersection(set(subjects_max_pres_drug_set))
len(patientts_expired_with_max_pres_drug)

11001

## DRUG \rightarrow SUBJECT_ID_LIST

In [34]:
drug_subject_dict = dict()
loop_count = 0
for drug in drugs_set:
    subjects_with_drug = set(prescriptions[prescriptions["DRUG"] == drug]["SUBJECT_ID"])
    drug_subject_dict[drug] = subjects_with_drug
    loop_count += 1
    if (loop_count%100 == 0):
        print ("Drugs iterated:%i;" %loop_count)

Drugs iterated:100;
Drugs iterated:200;
Drugs iterated:300;
Drugs iterated:400;
Drugs iterated:500;
Drugs iterated:600;
Drugs iterated:700;
Drugs iterated:800;
Drugs iterated:900;
Drugs iterated:1000;
Drugs iterated:1100;
Drugs iterated:1200;
Drugs iterated:1300;
Drugs iterated:1400;
Drugs iterated:1500;
Drugs iterated:1600;
Drugs iterated:1700;
Drugs iterated:1800;
Drugs iterated:1900;
Drugs iterated:2000;
Drugs iterated:2100;
Drugs iterated:2200;
Drugs iterated:2300;
Drugs iterated:2400;
Drugs iterated:2500;
Drugs iterated:2600;
Drugs iterated:2700;
Drugs iterated:2800;
Drugs iterated:2900;
Drugs iterated:3000;
Drugs iterated:3100;
Drugs iterated:3200;
Drugs iterated:3300;
Drugs iterated:3400;
Drugs iterated:3500;
Drugs iterated:3600;
Drugs iterated:3700;
Drugs iterated:3800;
Drugs iterated:3900;
Drugs iterated:4000;
Drugs iterated:4100;
Drugs iterated:4200;
Drugs iterated:4300;
Drugs iterated:4400;
Drugs iterated:4500;


NameError: name 'drugs_subject_dict' is not defined

In [94]:
with open('../Data/Drugs/drug_subject_dict.pickle', 'wb') as pickler:
    pickle.dump(drug_subject_dict, pickler, protocol=pickle.HIGHEST_PROTOCOL)

with open('../Data/Drugs/drug_subject_dict.pickle', 'rb') as pickler:
    dict_check = pickle.load(pickler)

print(len(dict_check))

4525


## Deaths associated with a drug

In [37]:
drug_subjects_expired_dict = dict()
drug_subjects_expired_count_dict = dict()
for drug, subjectIDs in drug_subject_dict.items():
    drug_subjects_expired_dict[drug] = subjectIDs.intersection(set(subjects_expired))
    drug_subjects_expired_count_dict[drug] = len(drug_subjects_expired_dict[drug])


In [41]:
drug_subjects_expired_count_dict

{'Cepacol (Menthol)': 354,
 'albut': 1,
 'Aciphex': 1,
 'Dexmedetomidine': 225,
 'Naltrexone': 2,
 'Insulin Human Regular': 2516,
 'Byetta (exenatide)': 0,
 'Propoxyphene Nap.-Apap (N-100)': 141,
 'Imiquimod': 1,
 'Sulfasalazine EC': 2,
 'NEO*IV*Amphotericin B': 0,
 'XIBROM': 2,
 'permethrin': 2,
 'Idarubicin HCl': 15,
 'Morphine Sulfate (PF)': 3,
 'Relpax': 1,
 'Econazole 1% Cream': 0,
 'Fentora': 1,
 'Dimen': 1,
 'Meningococcal Vaccine': 21,
 'Sodium Chloride 23.4%': 35,
 'Blue-Green Algae (Spirulina)': 0,
 'Loraz': 1,
 'basis soap': 1,
 'Xopenex HFA': 13,
 'Deferoxamine (DFO) *IND*': 2,
 'Avodart  0.5MG': 1,
 'Mumps Skin Test': 4,
 'methadone': 1,
 'Indinavir Sulfate': 4,
 'Castor Oil': 0,
 'Anagrelide HCl': 1,
 'Albuterol Inhaler': 1102,
 'Ketorolac': 735,
 'Rasburicase': 8,
 'Glucagon Emergency': 1,
 '*NF* Sodium Chloride 5% (Hypertonic)': 1,
 'Acyclovir Ointment 5%': 34,
 'valproate': 1,
 'SYMLIN': 0,
 'Roxanol': 3,
 'Ciprofloxacin 0.3% Ophth Soln': 57,
 'Amino Acids 5%-Dextrose 

In [51]:
loop_count = 0
for key, value in sorted(drug_subjects_expired_count_dict.items(), key=operator.itemgetter(1), reverse=True):
        loop_count += 1
        if (loop_count > 10):
            break
        else:
            print(key, value)

Potassium Chloride 11001
Acetaminophen 10847
Sodium Chloride 0.9%  Flush 10670
Insulin 10212
Magnesium Sulfate 10074
Heparin 9999
D5W 9936
NS 9813
Iso-Osmotic Dextrose 9693
Morphine Sulfate 9030


#### The drugs associted with maximum number of deaths might also be used more. So, it makes sense to calculate the death to usage ratio (or percentage).

In [66]:
drug_pres_count = dict()
drugs_death_usage_ratio = dict()
loop_count = 0
for drug in drugs_set:
    no_of_used_times = len(set(prescriptions[prescriptions["DRUG"] == drug]["SUBJECT_ID"]))
    drug_pres_count[drug] = no_of_used_times
    no_of_deaths = drug_subjects_expired_count_dict[drug]
    ratio = float(no_of_deaths)/float(no_of_used_times)
    drugs_death_usage_ratio[drug] = ratio
    loop_count += 1
    if (loop_count%100 == 0):
        print ("Drugs with ratio calculated: %i" %loop_count)

Drugs with ratio calculated: 100
Drugs with ratio calculated: 200
Drugs with ratio calculated: 300
Drugs with ratio calculated: 400
Drugs with ratio calculated: 500
Drugs with ratio calculated: 600
Drugs with ratio calculated: 700
Drugs with ratio calculated: 800
Drugs with ratio calculated: 900
Drugs with ratio calculated: 1000
Drugs with ratio calculated: 1100
Drugs with ratio calculated: 1200
Drugs with ratio calculated: 1300
Drugs with ratio calculated: 1400
Drugs with ratio calculated: 1500
Drugs with ratio calculated: 1600
Drugs with ratio calculated: 1700
Drugs with ratio calculated: 1800
Drugs with ratio calculated: 1900
Drugs with ratio calculated: 2000
Drugs with ratio calculated: 2100
Drugs with ratio calculated: 2200
Drugs with ratio calculated: 2300
Drugs with ratio calculated: 2400
Drugs with ratio calculated: 2500
Drugs with ratio calculated: 2600
Drugs with ratio calculated: 2700
Drugs with ratio calculated: 2800
Drugs with ratio calculated: 2900
Drugs with ratio calcul

In [95]:
with open('../Data/Drugs/drug_pres_count.pickle', 'wb') as pickler:
    pickle.dump(drug_pres_count, pickler, protocol=pickle.HIGHEST_PROTOCOL)

with open('../Data/Drugs/drugs_death_usage_ratio.pickle', 'wb') as pickler:
    pickle.dump(drug_pres_count, pickler, protocol=pickle.HIGHEST_PROTOCOL)

In [68]:
drugs_death_usage_ratio

{'Cepacol (Menthol)': 0.3007646559048428,
 'albut': 1.0,
 'Aciphex': 0.25,
 'Dexmedetomidine': 0.1820388349514563,
 'Naltrexone': 0.6666666666666666,
 'Insulin Human Regular': 0.3061572158676077,
 'Byetta (exenatide)': 0.0,
 'Propoxyphene Nap.-Apap (N-100)': 0.4259818731117825,
 'Imiquimod': 1.0,
 'Sulfasalazine EC': 0.6666666666666666,
 'NEO*IV*Amphotericin B': 0.0,
 'XIBROM': 0.6666666666666666,
 'permethrin': 0.4,
 'dilantin': 0.0,
 'Vicodin ': 0.0,
 'Relpax': 1.0,
 'Econazole 1% Cream': 0.0,
 'Dimen': 1.0,
 'Meningococcal Vaccine': 0.35,
 'Sodium Chloride 23.4%': 0.7,
 'Blue-Green Algae (Spirulina)': 0.0,
 'Loraz': 1.0,
 'basis soap': 1.0,
 'Xopenex HFA': 0.5652173913043478,
 'Risperidone Long Acting Injection': 0.625,
 'Avodart  0.5MG': 1.0,
 'traMADOL': 0.5903361344537815,
 'methadone': 1.0,
 'Indinavir Sulfate': 0.5714285714285714,
 'Amoxicillin-Clavulanate Desensitization': 0.0,
 'Anagrelide HCl': 1.0,
 'Albuterol Inhaler': 0.3949820788530466,
 'Ketorolac': 0.16104294478527606,

#### Selecting the medicines with heighest death ratio might not be correct. For example, if a subject is prescribed a particula drug only once and if that subject dies then it would lead to a ratio of 1. But, we do not want such type of rare drugs to be included. Hence, choose the top 500 mostly prescribed drugs and, among them, choose the ones with highest death ratio.

In [76]:
most_pres_drugs_500 = dict()
loop_count = 0
for drug, count in sorted(drug_pres_count.items(), key=operator.itemgetter(1), reverse=True):
    loop_count += 1
    if (loop_count > 500):
        break
    else:
        most_pres_drugs_500[drug] = count

In [77]:
most_pres_drugs_500

{'0.45% Sodium Chloride': 1353,
 '0.83% Sodium Chloride': 774,
 '0.9% Sodium Chloride': 15026,
 '0.9% Sodium Chloride (Mini Bag Plus)': 4178,
 '1/2 NS': 6108,
 '5% Dextrose': 11998,
 '5% Dextrose (EXCEL BAG)': 1164,
 'AMP': 680,
 'AcetaZOLamide': 375,
 'AcetaZOLamide Sodium': 857,
 'Acetaminophen': 28821,
 'Acetaminophen (Liquid)': 2551,
 'Acetaminophen IV': 1615,
 'Acetaminophen w/Codeine': 673,
 'Acetaminophen-Caff-Butalbital': 710,
 'Acetazolamide Sodium': 413,
 'Acetylcysteine 20%': 4047,
 'Acyclovir': 1368,
 'Adenosine': 275,
 'Albumin 25% (12.5 g)': 355,
 'Albumin 25% (12.5g / 50mL)': 1857,
 'Albumin 25% (12.5gm)': 444,
 'Albumin 5% (12.5g / 250mL)': 1404,
 'Albumin 5% (25 g)': 305,
 'Albumin 5% (25g / 500mL)': 2167,
 'Albuterol': 3648,
 'Albuterol 0.083% Neb Soln': 10140,
 'Albuterol Inhaler': 2790,
 'Albuterol MDI': 453,
 'Albuterol Neb Soln': 1940,
 'Albuterol-Ipratropium': 2881,
 'Alendronate Sodium': 474,
 'Allopurinol': 1601,
 'Alprazolam': 995,
 'Alteplase': 351,
 'Altepla

In [88]:
loop_count = 0
high_deathRatio_usage_500 = dict()
for drug, count in sorted(most_pres_drugs_500.items(), key=operator.itemgetter(1), reverse=True):
    loop_count += 1
    if (loop_count > 500):
        break
    else:
        high_deathRatio_usage_500[drug] = drugs_death_usage_ratio[drug]

high_deathRatio_usage_500 = sorted(high_deathRatio_usage_500.items(), key=operator.itemgetter(1), reverse=True)
high_deathRatio_usage_500

[('Morphine Sulfate (Concentrated Oral Soln)', 0.8747390396659708),
 ('Scopolamine Patch', 0.8721407624633432),
 ('Cefepime', 0.830423940149626),
 ('Hyoscyamine', 0.8272506082725061),
 ('Caspofungin', 0.824468085106383),
 ('Voriconazole', 0.7938144329896907),
 ('Morphine Sulfate (Oral Soln.)', 0.7890222984562607),
 ('Nesiritide', 0.7867647058823529),
 ('Nitroglycerin Oint. 2%', 0.7610389610389611),
 ('Ceftazidime', 0.7569444444444444),
 ('Ambisome', 0.7569444444444444),
 ('Lansoprazole Oral Solution', 0.7477203647416414),
 ('Sevelamer', 0.7462068965517241),
 ('Albumin 25% (12.5 g)', 0.7436619718309859),
 ('Vasopressin', 0.7417840375586855),
 ('Prismasate (B32 K2)', 0.7385257301808067),
 ('DopAmine', 0.7299324831207802),
 ('Sulfameth/Trimethoprim', 0.727710843373494),
 ('Filgrastim', 0.7099236641221374),
 ('Albumin 25% (12.5gm)', 0.7094594594594594),
 ('Cosyntropin', 0.7050673000791766),
 ('Fludrocortisone Acetate', 0.70375),
 ('Lansoprazole Oral Suspension', 0.7030965391621129),
 ('DOB

In [97]:
drug_albut_usage_count = len(set(prescriptions[prescriptions["DRUG"] == "Roxanol"]["SUBJECT_ID"]))
drug_albut_usage_count

3

In [99]:
icu_stays = pd.read_csv(data_dir+"ICUSTAYS.csv.gz", compression="gzip")
icu_stays.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [ ]:
input_events_cv = pd.read_csv(data_dir+"INPUTEVENTS_CV.csv.gz", compression="gzip")
input_events_cv.head()